In [65]:
import inscopix_cnmfe
import isx
import os, glob
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd

In [82]:
def find_paths(root_path, endswith: str):
    files = glob.glob(
        os.path.join(root_path, "**", endswith), recursive=True,
    )
    return files

dst = "/media/rory/Padlock_DT/BLA_Analysis/PTP_Inscopix_#1/BLA-Insc-1/Pre-RDT RM/motion_corrected.tif"
cellset_path = "/media/rory/Padlock_DT/BLA_Analysis/PTP_Inscopix_#1/BLA-Insc-1/Pre-RDT RM/cnmfe_cellset.isxd"
cnmfe_cellset = isx.CellSet.read(cellset_path)
num_cells = cnmfe_cellset.num_cells
mouse = cellset_path.split("/")[6]
session = cellset_path.split("/")[7]


"""dst = src.replace(".isxd", ".tif")
try:
    isx.export_movie_to_tiff(src, dst)
except Exception as e:
    print(e)
    pass"""

'dst = src.replace(".isxd", ".tif")\ntry:\n    isx.export_movie_to_tiff(src, dst)\nexcept Exception as e:\n    print(e)\n    pass'

In [83]:
footprints, traces = inscopix_cnmfe.run_cnmfe(
    input_movie_path=dst, 
    output_dir_path='output', 
    output_filetype=0,
    average_cell_diameter=16,
    min_pixel_correlation=0.7,
    min_peak_to_noise_ratio=8,
    gaussian_kernel_size=4,
    closing_kernel_size=0,
    background_downsampling_factor=1,
    ring_size_factor=1.125,
    merge_threshold=0.3,
    num_threads=5,
    processing_mode=2,
    patch_size=80,
    patch_overlap=20,
    output_units=1,
)

In [84]:
import pandas as pd
df_footprint_traces = pd.DataFrame(np.transpose(traces), columns=[num for num in range(0, len(traces))])
print(df_footprint_traces.head())

         0         1         2          3         4         5         6   \
0  0.685538  5.876531  2.659517   9.335667  4.635203 -0.644283  1.721727   
1  1.571262  5.346201  1.887426   7.697803  1.996394 -0.532356  1.362117   
2  2.349840  4.738593  1.360779   7.319066  2.014481  0.718466  0.855347   
3  3.002005  6.850334  1.669703  15.418008  3.512583  0.121274  1.710395   
4  2.797478  5.571459  0.506132  14.396134  3.387493  0.822241 -0.929986   

         7         8         9   ...        48        49        50        51  \
0 -2.380634  4.322753  2.876704  ...  4.672603  3.154080 -0.250667 -1.408403   
1  0.624313  0.827209  3.726662  ...  7.269340  4.471030 -1.424166  0.292777   
2 -0.353305  0.717526  0.547960  ...  7.243671  2.640567 -1.157599 -0.513034   
3 -1.007025  5.271232  2.012608  ...  5.014731  3.815519 -1.457119 -0.202252   
4  0.631175  2.273231  1.996314  ...  5.241078  4.872539  1.802006 -1.074353   

         52         53        54        55         56         

In [85]:
dst_dir = f"/media/rory/Padlock_DT/BLA_Analysis/LongReg/CellReg/{mouse}/{session}"
os.makedirs(dst_dir, exist_ok=True)

num_matches = 0
# iterating over accepted cells
for i in range(num_cells):
    cell_status = cnmfe_cellset.get_cell_status(i)
    cell_name = cnmfe_cellset.get_cell_name(i)
    if str(cell_status) == "accepted":
        #print(cell_name)
        cell_trace = list(cnmfe_cellset.get_cell_trace_data(i))
        cell_trace.sort()
        #print("OG:", cell_trace[:5])

        for trace_idx, footprint_trace in enumerate(traces):
            footprint_trace = list(footprint_trace)
            footprint_trace.sort()
            #print(footprint_trace[:5])

            if cell_trace == footprint_trace:
                #print("CHOSEN", footprint_trace[:5])
                footprint_idx = trace_idx
                print(f"footprint {footprint_idx} matches accepted cell {cell_name}")
                # appending idxs of the list of traces from footprints
                # save footprint of same trace idx as tif
                img = Image.fromarray(footprints[footprint_idx])
                img.save(f"{dst_dir}/{cell_name}_footprint.tif")
                num_matches += 1
                break
                
print(num_matches)

footprint 0 matches accepted cell C00
footprint 1 matches accepted cell C01
footprint 2 matches accepted cell C02
footprint 6 matches accepted cell C06
footprint 7 matches accepted cell C07
footprint 8 matches accepted cell C08
footprint 9 matches accepted cell C09
footprint 10 matches accepted cell C10
footprint 11 matches accepted cell C11
footprint 16 matches accepted cell C17
footprint 17 matches accepted cell C18
footprint 19 matches accepted cell C20
footprint 20 matches accepted cell C21
footprint 21 matches accepted cell C22
footprint 22 matches accepted cell C23
footprint 23 matches accepted cell C24
footprint 26 matches accepted cell C28
footprint 27 matches accepted cell C29
footprint 30 matches accepted cell C32
footprint 32 matches accepted cell C35
footprint 36 matches accepted cell C39
footprint 37 matches accepted cell C40
footprint 39 matches accepted cell C42
footprint 41 matches accepted cell C44
footprint 44 matches accepted cell C47
footprint 45 matches accepted ce